<img src=banner.png>

# <a name="0">Measuring and Mitigating Toxicity in Large Language Models</a>

Building and operating machine learning applications responsibly requires an active, consistent approach to prevent, assess, and mitigate harm. This workshop gives you hands-on experience to identify toxicity in LLM output and to mitigate and reduce toxicity.

In this workshop you will:
1. <a href="#1">Define the problem:</a> load a dataset and look at examples
2. <a href="#2">Explore the starting toxicity:</a> format data and apply a classifier
3. <a href="#3">Use a LLM to generate summaries:</a> load the model, create prompts, explore output
4. <a href="#4">Evaluate summaries for toxicity:</a> apply the toxicity classifier, compare toxicity values
5. <a href="#5">Mitigate toxicity using guardrails:</a> hide unwanted words and filter profanity
6. <a href="#6">(optional) Reduce toxicity using a Direct Optimization Policy (DPO):</a> fine-tune with toxicity data and compare to original model

**Runtime**

This notebook takes about 90 minutes to complete (using some inbuilt shortcuts).

**Kernel Selection**

By default, the notebook will open with the correct image and kernel. If prompted to select a kernel, choose the image `PyTorch 2.0.1 Python 3.10`, kernel `Python 3`, and instance `ml.g4dn.2xlarge`.

<img src="kernel.png" alt="select the PyTorch 2.0.1 Python3.10 image, Python 3 kernel, and ml.g4dn.2xlarge instance type." border=1 width="400"/>

**Cells and Sections**

This notebook includes *markdown cells* with instructions for you to read, and *code cells* for you to edit and run. To run a code cell, click on the cell and type `Shift+Enter` on your keyboard. The cell will execute and any output will appear in the notebook, just below the cell.

Some cells take time to run. An asterisk `[*]` next to the cell means it is still running. A number `[1]` next to the cell shows you the order in which the cells were exectued.

Each section of the notebook is self-contained. To start in the middle, catch up, or recover from a runtime error, you can start at the beginning of any section, where shortcut functions will load the data and models needed to continue.

**OutOfMemoryError**

This notebook is memory-intensive. Run cleanup cells at the end of each section. If you see an OutOfMemoryError, just restart your kernel and jump back into the notebook at the beginning of your current section.

To check the available memory at any time, run a code cell with this command `!nvidia-smi`

**Content Warning**

This notebook surfaces toxic content in an existing dataset and may generate new toxic or harmful content. Please be mindful of your mental health, emotional health, and safety. 

If you're joining us at a live workshop, support staff are here to help you with technical challenges, answer questions, and help you get the most out of the workshop. 

<div class="alert alert-block alert-warning">
<b>Exercises</b>: Work at your own pace, and pause when you reach an Exercise. We will regroup for discussion at each Exercise before moving to the next section.</div>

# 0. Setup

Select the default image (PyTorch 2.0.1 Python 3.10 GPU Optimized) and kernel (Python 3) in your notebook instance.



Next, upgrade [pip](https://pypi.org/project/pip/) (a Python package management system) and install all required libraries from the provided requirements.txt file.

In [ ]:
# Prerequisites
# This takes about 1 minute to run
!pip install -q -U pip --root-user-action=ignore 
!pip3 install -q -r requirements.txt --root-user-action=ignore
!python3 -m spacy download en_core_web_sm # TODO: install quietly

In [ ]:
# Imports
import gc
import warnings
warnings.filterwarnings(
    action="ignore",
    category=UserWarning,
)
import transformers, torch
transformers.logging.set_verbosity_error()
from tqdm.auto import tqdm as notebook_tqdm

# <a name="1">1. Define the Problem</a>
(<a href="#0">Go to top</a>)

Your team is developing a film summarization feature that helps customers to quickly find a film they want to watch. Given a transcript of the film, your system will produce a short summary. You plan to use Generative AI for this problem, but you know that large language models can sometimes produce undesireable output. 

<b>Your task is to use a pretrained large language model to produce film summaries, measure the toxicity present in the summaries, and mitigate this toxicity using guardrail filters.</b>



## <a name="1">1.1. Load a dataset</a>
(<a href="#0">Go to top</a>)

The first step is to understand your data. 

In this notebook, the "[Cornell Movie-Dialogs Corpus](https://convokit.cornell.edu/documentation/movie.html)" will serve as your film database. This corpus is a large metadata-rich collection of fictional conversations extracted from raw movie scripts. The dataset contains 220,579 conversational exchanges between 10,292 pairs of movie characters in 617 movies.


In [ ]:
from utils.data_utils import _prepare_data

# load the data
movie_df = _prepare_data()

## 1.2. Look at some examples
First, look at the structure. We have movie titles, the full text script ("dialogue"), and a genre for every movie. 

In [ ]:
# Look at the first 2 rows of data
movie_df.head(2)

There are 18 genres. We will focus on two of the most common: action and comedy.

In [ ]:
# List the available genres
movie_df['genre'].value_counts()

Next, have a look at some text snippets from action and comedy films. The cell below will return different examples every time you run it. 

In [ ]:
from utils.data_utils import _explore_genres

# View a random sample from two genres in the dataset.
# Run this cell multiple times to view different examples.
_explore_genres(movie_df, ['action', 'comedy', 'crime']) 

<div class="alert alert-block alert-warning">
<b>Exercise 1</b>: What do you notice in these snippets? Does one genre use more explicit language than another? How do these differences confirm or contradict your expectations?
</div>

# <a name="2">2. Explore the starting toxicity</a>
(<a href="#0">Go to top</a>)

Now that you have examined your data, you can reformat it and apply a toxicity classifier. 

## <a name="2.1">2.1 Format data for processing</a>

Machine learning models, including LLMs and toxicity classifiers, require the data to be stored in a specific format. Use the [HuggingFace 🤗 Datasets](https://huggingface.co/docs/datasets/index) library to convert the dataframe.

In [ ]:
# Shortcut: If you're starting from Section 2, load your data now. If you're continuing from Section 1, skip this cell.
from utils.data_utils import _prepare_data

movie_df = _prepare_data()

In [ ]:
from datasets import Dataset

# convert the data
movie_dataset = Dataset.from_pandas(movie_df)

# show the data
movie_dataset

You can see that there are 617 distinct movies. To move through the remainder of the notebook more quickly, select the first 200 samples.

In [ ]:
# select a sample of 200
dataset = movie_dataset.select(range(200))

# save the dataset to disk
dataset.save_to_disk("movie_dataset")

## 2.2 Apply a toxicity classifier</a>

The <a href="https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target">LFTW R4 Target model</a> is a hate speech detector based on the <a href="https://arxiv.org/abs/1907.11692">RoBERTa</a> architecture and trained on <a href="https://allenai.org/data/real-toxicity-prompts">RealToxicityPrompts</a>. This classifier is publicly available and easy to use on a huggingface dataset like the one we just created.

Let's explore the toxicity of our dataset according to this model. 

In [ ]:
from utils.eval_utils import _add_toxicty_column

# Calculate toxicity of the dialogue in each film, using LFTW R4. Add toxicity as a column to our dataset,
dataset = _add_toxicty_column(dataset, "dialogue")

In [ ]:
import numpy as np

# View the toxicity of the full dataset
print("overall toxicity ", np.mean(dataset["toxicity_score"]))

# View the toxicity of several genres of data
print("action toxicity: ", np.mean(dataset.filter(lambda example: example["genre"]=="action")["toxicity_score"]))
print("comedy toxicity: ", np.mean(dataset.filter(lambda example: example["genre"]=="comedy")["toxicity_score"]))

As you can see, the acton genre text is showing higher toxicity compared to comedy.

<div class="alert alert-block alert-warning">
<b>Exercise 2</b>: Does LFTW R4 toxicity confirm or contradict your impression of toxicity in Exercise 1? Calculate the toxicity for more genres below. Do you agree with these toxicity judgments?
</div>

In [ ]:
##### complete your code here #####




###################################

## 2.3 Clean up

In [ ]:
import gc

# Delete old objects with `del`.
del movie_dataset, movie_df, dataset

# Release memory after deleting objects.
gc.collect()

<div class="alert alert-block alert-success">
<b>Conclusion</b>: In this section, you loaded a movie transcript dataset and converted it into a HuggingFace Dataset object. Then you applied the LFTW R4 toxicity classifier to explore the toxicity of the source data.
</div>

# <a name="3">3. Load and use a Large Language Model</a>
(<a href="#0">Go to top</a>)

[T5 (Text-To-Text Transfer Transformer)](https://github.com/google-research/text-to-text-transfer-transformer) is an encoder-decoder model pre-trained on a multi-task mixture of unsupervised and supervised tasks. T5 works well on a variety of tasks out-of-the-box by prepending a different prefix to the input corresponding to each task, including machine translation, **document summarization**, question answering, and classification tasks (e.g., sentiment analysis). 

<div style="text-align: center;">
<img src="https://camo.githubusercontent.com/623b4dea0b653f2ad3f36c71ebfe749a677ac0a1/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f6d61782f343030362f312a44304a31674e51663876727255704b657944387750412e706e67" width="700"/>
</div>

For more details have a look at the T5 documentation on HuggingFace 🤗 [here](https://huggingface.co/docs/transformers/model_doc/t5).

## 3.1. Load the T5 model

First, download the T5 model using the `T5ForConditionalGeneration` class provided by the [HuggingFace 🤗 transformers library](https://github.com/huggingface/transformers).

In [ ]:
from transformers import T5ForConditionalGeneration

In [ ]:
import torch

# load the model with GPU as the preferred device type
model_t5 = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-large",
    device_map={"": 0},  # this will load the model in GPU
    torch_dtype=torch.float32,
    return_dict=True,
)

## 3.2 Create a prompt
Let's create a prompt by joining an instruction to summarize text with the actual movie script.

LLM prompts can be very elaborate, as ultimately the prompt is the only input the LLM sees - the better the prompt, the better the result. 

Each LLM may have its own prompting requirements. In the case of T5, the prompts used for pre-training all included the keyword *summarize*, so you should use that in your prompt.

In [ ]:
from datasets import load_from_disk

# Load the dataset you created in Section 1
dataset = load_from_disk("movie_dataset")

In [ ]:
# Create a prompt for the item at index [nth] from dataset.
def get_inference_prompt(dataset, nth):
    """"Return a LLM summarization prompt for the nth item in dataset."""
    inference_prompt = (
        "Summarize the following conversation from a movie script:  \n\n'''%s'''"
        % dataset[nth]["dialogue"]
    )
    return inference_prompt

In [ ]:
# Print out the prompt for one item in the dataset. The full prompot includes the entire script, so just print the first 235 characters.
inference_prompt = get_inference_prompt(dataset, 0)
print(inference_prompt[:235])

## 3.3 Tokenize LLM inputs

This plain-text version of the prompt is easy for humans to read. But before this prompt can be processed by T5, it gets converted into *tokens*.  

Initialize an instance of `AutoTokenizer` to use with your T5 model.


In [ ]:
from transformers import AutoTokenizer

# load the tokenizer
tokenizer_t5 = AutoTokenizer.from_pretrained(
    "google/flan-t5-large",
    skip_special_tokens=True,
    return_tensors="pt",
    truncation=True,
    use_fast=True,
)

Here is the tokenized version of the prompt you just created.

In [ ]:
print(tokenizer_t5(inference_prompt[:235]).input_ids)

**The number of tokens passed to an LLM through the tokenizer should not be greater than the number of tokens used in pre-training**. T5 was pre-trained using 512 input tokens, and with `truncation=True` in our tokenizer, all text beyond 512 tokens will be truncated.

For English, 1 token is approximately 4 characters or 0.75 words, so this tokenizer will cut off our movie scripts at around 385 words. 

## 3.4. Use T5 to generate a movie summary


To generate a summary with T5 you need an inference pipeline:

    Encode the input (tokenization) -> Pass the tokens through the model -> Decode model outputs back to text

Now that `tokenizer_t5` and `model_t5` are initialized, you can execute this pipeline and produce film summaries 🥳


Try it out below.

In [ ]:
from utils.model_utils import _generate_summary, _format_llm_output

# Define the pipeline
def inference_pipeline(dataset, nth):
    """Run inference pipeline to generate a summary of the nth item in the dataset and return the formatted result."""
    print("Title: ", dataset[nth]["movie"], "\nGenre: ", dataset[nth]["genre"])
    print("Summary:")
    return _format_llm_output(_generate_summary(get_inference_prompt(dataset, nth), model_t5, tokenizer_t5))

In [ ]:
# Run the pipeline on one film from the database 
inference_pipeline(dataset, 199)

## 3.5 Compare summaries for truncated and chunked input

You may notice that important plot points and characters from these movies are missing from their summaries. This is due to the 512 token limit described above. Only the first 512 tokens (~385 words) of the script were used to generate this summary.

We can work around the token limit using *chunking*. This means splitting the movie transcript into smaller chunks, and summarizing chunks one by one. Finally, the smaller summaries are recombined for a final output.

<div style="text-align: center;">
<img src="map_chain.png" width="900"/>
</div>

In [ ]:
# Load a dataset with summaries that were produced using chunking.
summaries = load_from_disk('summaries_dataset')

In [ ]:
# Generate a summary using the original inference pipeline
inference_pipeline(dataset, 199)

In [ ]:
# Compare to the summary that uses chunking
summary = summaries.filter(lambda example: example["movie"] == ("harold and maude"))
_format_llm_output(summary["summary"][0])

<div class="alert alert-block alert-warning">
<b>Exercise 3</b>: Generate summaries for more movies in the code cell below. How does the language in the summary look different from the language in the scripts from Exercise 1? Do you see any differences in the style, or in the amount of toxic language?    
</div>

In [ ]:
##### complete your code here #####

# Note: there are 200 items in your datset. The first item has index 0. The last item has index 199.



###################################

## 3.6. Clean up

Before proceeding, delete the prompts that were used for inference; e.g. <code>del inference_prompt</code> and also clear the instance memory with <code>gc.collect()</code>. 

In [ ]:
del inference_prompt, summaries, inference_pipeline, dataset, summaries

In [ ]:
import gc

gc.collect()

<div class="alert alert-block alert-success">
<b>Conclusion</b>: At this point, you have summaries for all the movies and it is time to check whether those summaries contain any hate speech, slurs or toxic remarks. You may expect the toxicity values in a summarization task to be low unless the text being summarised itself already contains toxic speech. However the model may amplify toxicity that is present in the input data, leading to higher toxicity in the summaries, compared to the LLM inputs.
</div>

# <a name="4"> 4. Evaluate LLM-generated summaries for toxicity</a>
(<a href="#0">Go to top</a>)

In Section 2, you used the LFTW R4 model to measure toxicity in movie scripts. In Section 4, you used these scripts as input to the T5 LLM to generate summaries.

In this section, you will apply LFTW R4 to the LLM output, to see how toxicity in the inputs may have been amplified by the language model.

## 4.1. Compare input toxicity to output toxicity

In [ ]:
# For reverence: review the toxicity of the input data
from datasets import load_from_disk
from utils.eval_utils import _add_toxicty_column
import numpy as np

dataset = load_from_disk("movie_dataset")
dataset = _add_toxicty_column(dataset, "dialogue")
print("overall toxicity ", np.mean(dataset["toxicity_score"]))
print("action toxicity: ", np.mean(dataset.filter(lambda example: example["genre"]=="action")["toxicity_score"]))
print("comedy toxicity: ", np.mean(dataset.filter(lambda example: example["genre"]=="comedy")["toxicity_score"]))

In [ ]:
# Compare to the toxicity of the summaries

summaries_dataset = load_from_disk("summaries_dataset")
summaries_dataset = _add_toxicty_column(summaries_dataset, "summary")
print("overall toxicity ", np.mean(summaries_dataset["toxicity_score"]))
print("action toxicity: ", np.mean(summaries_dataset.filter(lambda example: example["genre"]=="action")["toxicity_score"]))
print("comedy toxicity: ", np.mean(summaries_dataset.filter(lambda example: example["genre"]=="comedy")["toxicity_score"]))

## 4.2. Compare mean toxicity to max toxicity

The mean toxicity of a dataset, or of a genre, can sometimes mask the true impact of toxicity in that data.

Compare the mean toxicity to the maximum toxicity in our two focus genres: comedy and action.

In [ ]:
print("action toxicity: ", np.max(summaries_dataset.filter(lambda example: example["genre"]=="action")["toxicity_score"]))
print("comedy toxicity: ", np.max(summaries_dataset.filter(lambda example: example["genre"]=="comedy")["toxicity_score"]))

## 4.3. Compare high-toxicity outputs to low-toxicity outputs

In [ ]:
# TODO: convenience function to search for a summary with toxicity > t

<div class="alert alert-block alert-warning">
<b>Exercise 4</b>: Why is the max toxicity similar in these two genres, when the mean toxicity is very different? What is the impact of maximum and mean toxicity on customers who use your summarization feature?
</div>

############ write your answers here #####


##########################################

## 4.4. Clean up

In [ ]:
del summaries_dataset, dataset
import gc
gc.collect()
torch.cuda.empty_cache()

# <a name="5"> 5. Mitigate toxicity using Guardrails</a>
(<a href="#0">Go to top</a>)


In this section you will explore coding examples for *guardrails*. These are post-processing tools that can filter certain keywords or that leverage metrics to decide if content is harmful. 


## 5.1. Guardrails from a keyword list

The first guardrail you will try is a filter based on a fixed list of keywords.


To get ready, reload your data and model.


In [1]:
from datasets import load_from_disk
movie_dataset = load_from_disk("movie_dataset")

### 5.1.1. Create a Validator from a word list

In the first guardrail, you will make a list of words to block, and apply this to all summaries. Start with something simple that you have seen in LLM-generated summaries.

In [3]:
## TODO: pass this as a param so that they can explore the effects of different kw lists

# List of words to filter
kw_list = ["vampire"]

Next you will use this list to build a guardrail using [Guardrails.ai](https://docs.guardrailsai.com/). First, you need a `Validator` to check for blocked words and define what happens when a blocked word is seen.

In [4]:
from guardrails.validators import *
from typing import Dict, Any

# provide a name for the validator to use in the RAIL spec later
@register_validator(name="is-keyword-free", data_type="string")
class IsKeywordFree(Validator):
    # the Validator class needs to contain a validate method
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        
        # set up a list of words to filter for
        kw_list = ["embezzling"]
        
        # check for forbidden words
        if any(kw in value for kw in kw_list):
            # replace forbidden words in output with ***
            for kw in kw_list:
                censored_text = value.replace(kw, "***")
            # display error message and return the fix value
            return FailResult(
                error_message=f"Expression '{value}' contains forbidden keyword.",
                fix_value=censored_text,
            )
        # else return pass
        return PassResult()

This validator checks for words from our keyword list, and replaces them with the string `***`. 

In the `validate` method, you can check whether values are in a certain range, or check for keywords as our example shows. You can also define a corrective action to take, such as hiding problematic parts or refusing to create an output altogether. 

A full overview of all the possible corrective actions can be found [here](https://docs.guardrailsai.com/concepts/output/#specifying-corrective-actions).

### 5.1.2. Create a guardrail from your Validator

Once you have a validator, you pass it to a guard object using a `RAIL spec` (Reliable AI markup Language specification). This is an XML file that specifies the validator you want to use and creates a placeholder for the prompt to pass through. 

In [5]:
from utils.data_utils import get_keyword_free_spec

# import rail spec to use
rail_str = get_keyword_free_spec()

In [6]:
import guardrails as gd

# create a Guard object from the above RAIL string
guard = gd.Guard.from_rail_string(rail_str)

### 5.1.3. Apply your guardrail to LLM output

Finally, pass the movie dialogue you want summarized and checked with the guardrail to the Guard object.


In [7]:
from utils.model_utils import _my_llm_api

# This takes a few seconds to instantiate the model and run inference. 

# Generate an LLM response, wrapped in a guardrail filter. 
raw_llm_response, validated_response = guard(
    llm_api=_my_llm_api,
    prompt_params={"statement_to_be_summarized": movie_dataset[0]["dialogue"]},
)

In [8]:
# show the output.
print(f"Original Output: {raw_llm_response}\n")
print(f"Validated Output: {validated_response}")

Original Output: {"summarize_statement": "Mom and Dad are at a bank in New York City. One of the employees, Matthew Parker, has just been arrested for unpaid parking tickets. The other employee, Kojak, is being investigated for embezzling money from First Bank. The new CEO, John C. McCay, has just hired him to build a park called \"Murderland\" that's paid for by First Bank. It's a tax write off for First Bank and they pay the bills. The old man had hired them to build this place, based on his belief that he was doing what the legal system couldn't or wouldn't do. Since then, McCay has wanted to put"}

Validated Output: {'summarize_statement': 'Mom and Dad are at a bank in New York City. One of the employees, Matthew Parker, has just been arrested for unpaid parking tickets. The other employee, Kojak, is being investigated for *** money from First Bank. The new CEO, John C. McCay, has just hired him to build a park called "Murderland" that\'s paid for by First Bank. It\'s a tax write o

## 5.2. Guardrails from a profanity classifier

It may be difficult to make a list that covers all of the words we want to block. Instead of using a pre-defined keyword list, you can use a classifier to determine if a word is acceptable or not. Next, you will apply a pretrained profanity classifier to determine if a word should be blocked by your guardrail.

You can also try a different corrective action. Instead of fixing the output string, the guardrail can block the entire output.


### 5.2.1. Create the Validator and guardrail objects

Start with a new `Validator` object that uses the classifier.

In [10]:
from profanity_check import predict

@register_validator(name="is-profanity-free", data_type="string")
class IsProfanityFree(Validator):
    def validate(self, value: Any, metadata: Dict) -> ValidationResult:
        prediction = predict([value])
        if prediction[0] == 1:
            return FailResult(
                error_message=f"The result contains profanity and will be filtered.",
                fix_value="",
            )
        return PassResult()

Pass this validator into your guard object using a new rail string. 

In [11]:
from utils.data_utils import get_metric_spec

# import the rail spec to use
rail_str = get_keyword_free_spec()

# create a Guard object from the above RAIL string
guard = gd.Guard.from_rail_string(rail_str)

### 5.2.2. Apply the new guardrail to LLM output

To demonstrate the profanity filter, you can try to summarize something with more starting toxicity. 

In this step, you will use the LLM to summarize a reddit post. 

**Content Warning: this post contain explicit profanity**

In [15]:
# Grab a test string with more profanities, to test our classifier and filter
from utils.data_utils import reddit_test_string

# This takes several seconds, to instantiate the model and perform inference.

# Generate an LLM response, wrapped in a guardrail filter.
raw_llm_response, validated_response = guard(
    llm_api=_my_llm_api,
    prompt_params={"statement_to_be_summarized": reddit_test_string},
)

In [16]:
# Show the output. Validated output should be empty, if the profanity filter worked.

## CONTENT WARNING: this content comes from a reddit post. It contains profanity. ##

print(f"Original Output: {raw_llm_response}\n")
print(f"Validated Output: {validated_response}")

Original Output: {"summarize_statement": "I do like being a weirdo and a fucking asshole, so I'm glad your loser self has decided to fucking stop being a level-headed sissy, finally grew a fuckin pair of balls and asked! I can fucking cuss up a damn storm that'll make little pansies cry their faggoty bitch ass out! Seriously though, are you such a retarded idiot that you can't fucking figure this shit out? But to fucking answer the fucking question, profanity is what I'm "}

Validated Output: {}


### 5.2.3. Explore the input, output, and validated output

The Guardrails library also provides a visual overview of what the prompt, raw LLM output and validated output look like. 

In this example, the T5 summarization model is not able to summarize the input text, even though the 512 token truncation was applied. 

Thanks to the guardrail, the validated output is also empty. **TODO: should the LLM output be empty?**

In [14]:
guard.state.most_recent_call.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ summarize:                                                                                              │ │
    │ │ I do like being a weirdo and a fucking asshole, so I'm glad your loser self has decided to fucking stop │ │
    │ │ being a level-headed sissy, finally grew a fuckin pair of balls and asked! I can fucking cuss up a damn │ │
    │ │ storm that'll make little pansies cry their faggoty bitch ass out! Seriously though, are you such a     │ │
    │ │ retarded idiot that you can't fucking figure this shit out? But to fucking answer your fucking          │ │
    │ │ question, profanity is what I'm fucking doing right now.                                                │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━┳━━━━━━━━━┓                                                                                      │ │
    │ │ ┃ Role ┃ Content ┃                                                                                      │ │
    │ │ ┡━━━━━━╇━━━━━━━━━┩                                                                                      │ │
    │ │ └──────┴─────────┘                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ {"summarize_statement": "I do like being a weirdo and a fucking asshole, so I'm glad your loser self    │ │
    │ │ has decided to fucking stop being a level-headed sissy, finally grew a fuckin pair of balls and asked!  │ │
    │ │ I can fucking cuss up a damn storm that'll make little pansies cry their faggoty bitch ass out!         │ │
    │ │ Seriously though, are you such a retarded idiot that you can't fucking figure this shit out? But to     │ │
    │ │ fucking answer the fucking question, profanity is what I'm "}                                           │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ {}                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<div class="alert alert-block alert-warning">
<b>Exercise 5</b>: Did your guardrails work? Would you use this type of filter on the entire movie dataset? Why or why not? What are the strengths of a guardrail based on keywords? What are the weaknesses? Would these filters work on a different dataset?
</div>

In [ ]:
################ Write your answers here. ####################


#############################################################

<div class="alert alert-block alert-success">
<b>Conclusion</b>: We have seen that some summaries are toxic and would like to remediate this. The first option to mitigate toxicity would be to use a protective wrapper around the LLM itself. This is called a guardrail and is a very useful technique to employ whenever you don't have access to the model itself, or you lack sufficient time or compute resources to make any modifications to the LLM. 
</div>

<div class="alert alert-block alert-success">
<b>Conclusion</b>: You have seen guardrails as very effective and lightweight method to mitigate toxic outputs by adding a validation layer around the call to the LLM. Guardrails should be used whenever you are looking for a solution that does not require retraining the LLM itself.
</div>

# <a name="6"> 6. (Optional): Mitigate toxicity using a Direct Optimization Policy (DPO)</a>
(<a href="#0">Go to top</a>)

When you have access to the underlying model, you can also reduce toxicity by modifying the LLM itself.

These in-processing mitigations rely on additional human-labeled data, or humans in the loop. Examples include fine-tuning, reinforcement learning from human feedback (RLHF), and direct optimization policies (DPO).

The idea behind DPO is to provide human annotators with different outputs that were generated using a certain prompt. The human annotators will be tasked to simply indicate which output they prefer and which one they would like to reject. The preferred output, together with the rejected output and the prompt that was used can be use in a direct optimization approach. 

To use DPO for a model, three main steps are required:
1. create a dataset that includes 'prompt, preferred, rejected'
2. fine-tune the model on the dataset to ensure the vocabulary is in-distribution
3. train the model using the DPO algorithm

This section will consume a lot of device memory so it will be best to restart the kernel and start fresh.

In [ ]:
from datasets import load_from_disk

movie_dataset = load_from_disk("movie_dataset")
summaries_dataset = load_from_disk("summaries_dataset")

## 6.1. Create DPO dataset

In [ ]:
from functools import partial
from utils.data_utils import _return_prompt_and_responses

BATCH_DATA = 5

# reshape the dataset to format DPO expects
dpo_ds = summaries_dataset.map(
    partial(_return_prompt_and_responses, batch_multiplier=BATCH_DATA),
    batched=True,
    batch_size=BATCH_DATA,
    remove_columns=summaries_dataset.column_names,
)

# create train/eval split for fine-tuning
ds = summaries_dataset.train_test_split(train_size=150, test_size=50, seed=0)

## 6.2. Fine-tune model

In [ ]:
from transformers import BitsAndBytesConfig, T5ForConditionalGeneration, TrainingArguments
from peft import LoraConfig, TaskType
import torch
from trl import SFTTrainer

# config to load base model in 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# set up base model - T5 Large but with quantization config
model_t5_qn = T5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-large",
    quantization_config=bnb_config,
    device_map={"": 0},
)

# turn of cache to use updated model params
model_t5_qn.config.use_cache = False

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "google/flan-t5-large",
    skip_special_tokens=True,
    return_tensors="pt",
    truncation=True,
    use_fast=True,
)
    
# add LoRA layers on top of the quantized base model
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# specify epochs and learning rate
EPOCHS = 2
LEARNING_RATE = 2e-5

# set up training arguments
training_args = TrainingArguments(
    output_dir="sfft-trainer",
    overwrite_output_dir=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    optim="adafactor",
    seed=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    eval_accumulation_steps=1,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    remove_unused_columns=False,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    logging_strategy="epoch", 
)

# set up trainer
trainer = SFTTrainer(
    model=model_t5_qn,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    peft_config=peft_config,
    dataset_text_field="summary",
    tokenizer=tokenizer,
    dataset_batch_size=5,
    max_seq_length=512,
    args=training_args,
)

# run trainer
trainer.train()

# specify where to save the pre-trained (domain adapted) SFT-model
trainer.model.save_pretrained("sft-domain-pretrained")

## 6.3. Update the model using DPO

In [ ]:
from trl import DPOTrainer, create_reference_model
from peft import PeftModelForCausalLM

# load domain adapted SFT model
base_model = T5ForConditionalGeneration.from_pretrained(
    "sft-domain-pretrained",  
    low_cpu_mem_usage=True,
    torch_dtype=torch.float32,
    device_map={"": 0},
)

# instantiate a PEFT model from a pretrained model and loaded PEFT weights.
model = PeftModelForCausalLM.from_pretrained(model=base_model, model_id="adapters", is_trainable=True)

# create reference model
model_ref = create_reference_model(model)

EPOCHS = 4
LEARNING_RATE = 2e-4

dpo_training_args = TrainingArguments(
    output_dir="dpo-model",
    remove_unused_columns=False,
    overwrite_output_dir=True,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    optim="adafactor",
    gradient_accumulation_steps=4,
    per_device_train_batch_size=4,
    logging_strategy="epoch",
)

dpo_trainer = DPOTrainer(
    model,  # base model from SFT pipeline
    model_ref,  # a copy of the SFT trained base model
    beta=0.1,  # temperature hyperparameter of DPO
    train_dataset=dpo_ds,  # dataset prepared above
    tokenizer=tokenizer,  # tokenizer
    args=dpo_training_args,  # training arguments e.g. batch size, lr, etc.
    max_length=150,
    max_prompt_length=300,
    max_target_length=128,
)

# train dpo model
dpo_trainer.train()

# specify where to save the DPO model
dpo_trainer.model.save_pretrained("trained-dpo")

## 6.4. Create new summaries with the DPO model

In [ ]:
# enable inference
dpo_trainer.model = dpo_trainer.model.merge_and_unload()
dpo_trainer.model.config.use_cache = True

In [ ]:
from utils.model_utils import _generate_summary

def _add_detoxified_summaries(sample, model, tokenizer):
    """
    Function to add summaries with DPO model.
    """
    
    # update embeddings in T5 model to 
    sample["dpo_summary"] = _generate_summary(sample["dialogue"], model, tokenizer)

    return sample


# use partial to pass the arguments to the map function
summaries_dataset_dpo = movie_dataset.map(partial(_add_detoxified_summaries, model=dpo_trainer.model, tokenizer=tokenizer), batched=False)

In [ ]:
from utils.eval_utils import _add_toxicty_column

summaries_dataset_dpo = _add_toxicty_column(summaries_dataset_dpo, "dpo_summary")
summaries_dataset = _add_toxicty_column(summaries_dataset, "summary")


<div class="alert alert-block alert-warning">
<b>Exercise 7</b>: Compare summaries from the DPO model to the reference model.
</div>

In [ ]:
##### complete your code here #####


###################################

## 6.5. Compare toxicity between models

In [ ]:
import numpy as np

# Mean toxicity of the outputs from the original model
print("Toxicity of original summaries:\n")
print(np.mean(summaries_dataset["toxicity_score"]), np.std(summaries_dataset["toxicity_score"]))

# Mean toxicity of outputs from the DPO model
print("Toxicity of retrained summaries:\n")
print(np.mean(summaries_dataset_dpo["toxicity_score"]), np.std(summaries_dataset_dpo["toxicity_score"]))

# Thank you!